# Data Discovery - Investigando os Dados Brutos

**Autor:** Felipe Freire (Diretor de operações)
**Objetivo:** Este notebook serve como um ambiente de exploração para investigar os dados brutos, especificamente os valores únicos de colunas categóricas que precisam de limpeza e padronização.

**Workflow:**
1. Carregar o dataset **bruto** (`raw`).
2. Para cada coluna de interesse (`estado`, `etnia`, `genero`, etc.), usar `.unique()` e `.value_counts()` para listar todas as variações existentes.
3. Usar a saída desta análise para preencher os dicionários de mapeamento no script principal `src/analysis.py`.

In [ ]:
import pandas as pd
import os

# --- Configuração dos Caminhos ---
# Esta célula garante que o notebook encontre os arquivos de dados corretamente
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..')) # Sobe um nível para a pasta raiz do projeto
RAW_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'raw', '20250916-div_inscricoes.csv')

# --- Carregando os Dados ---
# Sempre carregamos os dados brutos aqui para garantir que estamos vendo a "verdade" original
print(f"Carregando dados de: {RAW_DATA_PATH}")
df_raw = pd.read_csv(RAW_DATA_PATH)

print("\nDados carregados com sucesso!")
df_raw.head()

In [6]:
pip uninstall pandasql
pip uninstall sqlite3 # Tente desinstalar, mas se for um módulo nativo do Python, pode falhar.

SyntaxError: invalid syntax (45909050.py, line 1)

In [16]:
import pandas as pd
import duckdb

# df_raw deve estar carregado aqui...

consulta_sql = """
    SELECT
        turma_slug,
        curso_titulo
    FROM
        df_raw  -- O DuckDB lê DataFrames como se fossem tabelas
    
"""

# 1. Conecta ao DuckDB na memória
conn = duckdb.connect()

# 2. Executa a consulta e armazena o resultado em um novo DataFrame
resultado_df = conn.execute(consulta_sql).fetchdf()

conn.close() # Fecha a conexão

print("\nResultado da Consulta SQL com DuckDB:")
print(resultado_df)

BinderException: Binder Error: Referenced column "curso_titulo" not found in FROM clause!
Candidate bindings: "outra_coluna", "turma_slug", "valor_coluna"

LINE 4:         curso_titulo
                ^

---
### Análise da Coluna `estado`

Vamos inspecionar os valores da coluna de estado para identificar inconsistências (ex: "SP" vs "São Paulo").

In [7]:
# .value_counts() é excelente pois mostra os valores e a frequência de cada um
print("Valores únicos e suas contagens para a coluna 'estado':")
print(df_raw['cidade'].unique())

Valores únicos e suas contagens para a coluna 'estado':
['Três passos' 'Triunfo' 'São Paulo' 'Osasco' 'Cabedelo' 'Salvador'
 'Rio de Janeiro' nan 'rio de janeiro' 'Matosinhos' 'Porto alegre'
 'Joinville' 'Valparaiso' 'Recife' 'Santa Luzia' 'Jaú' 'Manaus'
 'Porto Velho' 'Natal' 'Belo Horizonte' 'Malacacheta'
 'Itapecerica da Serra' 'Sabará' 'Maceió' 'Brasília' 'Vila Velha'
 'Angra dos Reis' 'Sumaré' 'Nova Iguaçu' 'Suzano' 'Vitória da Conquista'
 'Guarapuava' 'Campinas' 'Florianópolis' 'João Pessoa' 'Porto Alegre'
 'Senhor do Bonfim' 'Cachoeirinhas' 'Porto' 'Rio Branco' 'Guarulhos'
 'Juiz de Fora' 'Blumenau' 'Betim' 'Sapiranga' 'Belém' 'Niterói'
 'Anápolis' 'Ribeirão Preto' 'Araucária' 'Fortaleza' 'Bela Vista'
 'São João del Rei' 'São Leopoldo' 'Rio de janeiro' 'Alfenas' 'Maua'
 'Itaquaquecetuba' 'Presidente Prudente' 'Sertãozinho' 'Diamantina'
 'Maracanaú' 'Vila Sônia' 'Jaguaruana' 'Canoas' 'Nova Lima'
 'Três Corações' 'São Bernardo do Campo' 'Curitiba' 'Rio Claro'
 'Trancoso, Porto Seg

---
### Análise da Coluna `etnia`

Aqui, procuramos por variações como "etnia_branca", "[branca]", "Branca", além de valores nulos.

In [4]:
print("Valores únicos e suas contagens para a coluna 'etnia':")
print(df_raw['etnia'].value_counts(dropna=False)) # dropna=False também mostra a contagem de valores nulos (NaN)

Valores únicos e suas contagens para a coluna 'etnia':
etnia
etnia_branca                 657
NaN                          413
etnia_parda                  371
etnia_preta                  185
[]                           171
["branca"]                   134
["parda"]                     49
["preta"]                     37
etnia_indigena                28
etnia_amarela                 22
etnia_nao_quero_responder     12
etnia_outro                    8
["amarela"]                    6
["preta","parda"]              4
["indigena"]                   1
["nao_quero_responder"]        1
Name: count, dtype: int64


---
### Análise da Coluna `genero`

Investigamos a padronização das identidades de gênero declaradas.

In [5]:
print("Valores únicos e suas contagens para a coluna 'genero':")
print(df_raw['genero'].value_counts(dropna=False))

Valores únicos e suas contagens para a coluna 'genero':
genero
identidade_trans                           933
NaN                                        386
identidade_nao_binarie                     234
[]                                         174
["trans"]                                  164
identidade_cis                              87
["nao_binarie"]                             23
identidade_outro                            17
identidade_queer                            16
identidade_nao_sei                          15
["cis"]                                      8
identidade_nao_quero_responder               8
["trans","travesti"]                         7
["nao_binarie","queer"]                      5
["trans","nao_binarie"]                      5
["outro","Transmasculino"]                   4
["travesti"]                                 2
["intersexo - 46 x,y"]                       2
["Homem"]                                    1
["transmasculino","trans"]                  

In [6]:
# --- Análise da Coluna `atuacao` (Voluntariado) ---
print("\n" + "="*50)
print("Analisando dados de voluntariado para a coluna 'atuacao'")

# Carrega o dataset de voluntariado
RAW_VOLUNTARIADO_PATH = os.path.join(PROJECT_ROOT, 'data', 'raw', '20250916-div_voluntariado.csv')
df_vol_raw = pd.read_csv(RAW_VOLUNTARIADO_PATH)

print("\nValores únicos e suas contagens para a coluna 'atuacao':")
print(df_vol_raw['atuacao'].value_counts(dropna=False))


Analisando dados de voluntariado para a coluna 'atuacao'

Valores únicos e suas contagens para a coluna 'atuacao':
atuacao
["tecnologia"]                                                     96
["comunicacao"]                                                    23
["psicologia"]                                                     21
["parcerias"]                                                      15
["engajamento"]                                                    12
["outras"]                                                          5
["comunicacao","tecnologia"]                                        4
["comunicacao", "tecnologia"]                                       3
["tecnologia","outras"]                                             2
["tecnologia","comunicacao","outras"]                               2
["tecnologia","comunicacao"]                                        2
["engajamento","comunicacao","tecnologia"]                          1
["tecnologia","engajamento","parceri

In [7]:
# Adicione ao seu notebook para investigar
print("\n" + "="*50)
print("Verificando dados para o cruzamento de Gênero vs. Nível Profissional")

# Carrega o dataset final e processado
df_final = pd.read_csv('../data/processed/dados_consolidados_comunidade.csv')

# Cria uma tabela resumo
resumo_cruzamento = df_final.groupby('genero_padronizado').agg(
    total_de_pessoas=('person_id', 'count'),
    com_nivel_profissional=('professional_level_padronizado', 'count') # 'count' ignora valores nulos (NaN)
).reset_index()

print("\nComparativo de preenchimento:")
print(resumo_cruzamento)


Verificando dados para o cruzamento de Gênero vs. Nível Profissional



Comparativo de preenchimento:
      genero_padronizado  total_de_pessoas  com_nivel_profissional
0              Cisgênero                96                       0
1       Outra identidade                19                       0
2           Pessoa Trans              1118                      69
3  Preferiu não informar               843                     166
4                  Queer                23                       0
5               Travesti                 2                       1
